# this notebook is for Week5-Assignment-all figures
# author: Rui Zeng, N15471955, rz2522@nyu.edu
# assignment:Make a plot of phi vs time and psi vs time.
# Make a free energy surface in phi, psi space from this data. In otherwords, compute a 2d histogram of the phi, psi data and plot the -log(histogram), nicely labeled, with a colorbar, etc
# Make a free energy surface for T=300 using the data in T=300. Do you now see the whole surface? Set the maximum free energy to 10 kT
# Plot how each replica is moving in temperature in replica_temp (you may have to do every 100 frame to see what's happening) and make sure it goes up and down
# Using 'gmx_mpi energy', calculate the potential energy in each replica. Compute a histogram of energy in each case, and plot all the histograms on the same plot. Do they overlap well?

In [1]:
# import libraries
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load the trajectory from ../Data
trr_file = '../Data/adp.trr'
gro_file = '../Data/adp.gro'
traj = md.load(trr_file, top=gro_file)
# check the trajectory
print(traj)

<mdtraj.Trajectory with 5001 frames, 22 atoms, 3 residues, and unitcells>


In [3]:
# compute phi and psi angles
phi = md.compute_phi(traj)
psi = md.compute_psi(traj)

In [ ]:
# figure 1: phi & psi over time
fig=plt.figure(1)
# 
